In [1]:
%load_ext autoreload
%autoreload 2

import os
import time
import glob
import re
import pandas as pd
from functools import reduce
from klepto.archives import dir_archive

import numpy as np
from tqdm.auto import tqdm
import coffea.processor as processor
from coffea.processor.accumulator import AccumulatorABC
from coffea.analysis_objects import JaggedCandidateArray
from coffea.btag_tools import BTagScaleFactor
from coffea import hist
import pandas as pd
import uproot_methods
import uproot
import awkward
import copy

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from Tools.config_helpers import *
from Tools.helpers import mergeArray, mt, get_scheduler_address

from Tools.objects import Collections
from Tools.cutflow import Cutflow

# This just tells matplotlib not to open any
# interactive windows.
matplotlib.use('Agg')

In [2]:
# Imports and defintions for the actual analysis to run

from Tools.WH_objects       import *
from Tools.WH_scalefactors  import LeptonSF
from Tools.WH_deepAK8       import getWTagSF
from Tools.WH_signalWeights import getSignalWeight
from Tools.WH_samples       import * 

In [3]:
processesList = ['TTJets_old','TTJets_new']

plotDir = '/home/users/ksalyer/public_html/dump/WH_had/'

year = 2018

In [9]:
class analysisProcessor(processor.ProcessorABC):
    """Processor used for running the analysis"""
    def __init__(self):
        
        # we can use a large number of bins and rebin later
        dataset_axis        = hist.Cat("dataset",   "Primary dataset")
        pt_axis             = hist.Bin("pt",        r"$p_{T}$ (GeV)", 1000, 0, 1000)
        p_axis              = hist.Bin("p",         r"$p$ (GeV)", 1000, 0, 2500)
        ht_axis             = hist.Bin("ht",        r"$H_{T}$ (GeV)", 500, 0, 5000)
        mass_axis           = hist.Bin("mass",      r"M (GeV)", 1000, 0, 2000)
        eta_axis            = hist.Bin("eta",       r"$\eta$", 60, -5.5, 5.5)
        delta_axis          = hist.Bin("delta",     r"$\delta$", 100,0,10 )
        multiplicity_axis   = hist.Bin("multiplicity",         r"N", 20, -0.5, 19.5)
        norm_axis           = hist.Bin("norm",         r"N", 25, 0, 1)

        self._accumulator = processor.dict_accumulator({
            "met":          hist.Hist("Counts", dataset_axis, pt_axis),
            
            'TTJets_old':   processor.defaultdict_accumulator(int),
            'TTJets_new':   processor.defaultdict_accumulator(int),
        })

    @property
    def accumulator(self):
        return self._accumulator

    def process(self, df):
        """
        Processing function. This is where the actual analysis happens.
        """
        output = self.accumulator.identity()
        dataset = df["dataset"]
        cfg = loadConfig()
        
        ## correct x-sec for signal
        if dataset.count('TChiWH'):
            signalWeight = getSignalWeight(df, dataset, year=year)
            df['weight'] = signalWeight
            #signal_xsec[dataset]['xsec'] / signal_xsec[dataset]['sumweight']
        
        ## MET -> can switch to puppi MET
        met_pt  = df["MET_pt"]  if not year==2017 else df["METFixEE2017_pt"]
        met_phi = df["MET_phi"] if not year==2017 else df["METFixEE2017_phi"]
        
        ## Load Objects
        muon     = getMuons(df, WP='veto')
        electron = getElectrons(df, WP='veto')
        tau      = getTaus(df)
        isotrack = getIsoTracks(df)
        fatjet   = getFatJets(df)
        jet      = getJets(df)
        
        triggers = getTriggers(df, year=year, dataset=dataset) 
        filters  = getFilters(df, year=year, dataset=dataset)
        
        sf = LeptonSF(year=year)
        leptonSF = sf.get(electron, muon)
        
        stitch = df['stitch']
        
        ## Clean Objects
        skimjet   = jet[(jet.pt>30) & (jet.jetId>1) & (abs(jet.eta)<2.4)]
        jet       = jet[~jet.match(muon, deltaRCut=0.4)] # remove jets that overlap with muons
        jet       = jet[~jet.match(electron, deltaRCut=0.4)] # remove jets that overlap with electrons
        jet       = jet[jet.pt.argsort(ascending=False)] # sort the jets
        extrajet  = jet[~jet.match(fatjet, deltaRCut=0.8)] # remove AK4 jets that overlap with AK8 jets
        btag      = getBTags(jet, year=year)
        
        ## H-tagged Variables
        htag = getHTags(fatjet, year=year)        
        lead_htag = htag[htag.pt.argmax()]
        
        ## W-tagged Variables
        # deepAK8 working points: https://twiki.cern.ch/twiki/bin/viewauth/CMS/DeepAK8Tagging2018WPsSFs
        wtag = getWTags(fatjet, year=year)
        wtag = wtag[~wtag.match(htag, deltaRCut=0.8)]
        lead_wtag = wtag[wtag.pt.argmax()]
        
        if dataset.lower().count('data')==0:
            #print(dataset)
            GenW = getGenW(df)
            #print(GenW)
            wtag_SF = getWTagSF(wtag, GenW, year=year)

        ## variables for selection
        if dataset == 'TTJets_old':
            selection = ( (btag.counts==2) & (electron.counts+muon.counts==1) )
        else:
            selection = ( (btag.counts==2) & (electron.counts+muon.counts==1) & (stitch==1) )
       
        #output['totalEvents']['all'] += len(df['weight'])
        
        # Cutflow
        processes = processesList
        #weight = np.ones(len(df['weight'])) if dataset=='Data' else df['weight']
        #lumi = 1 if dataset=='Data' else 60.*LeptonSF
        #fullweight = weight * lumi
        weight      = np.ones(len(df['weight'])) if dataset=='Data' else df['weight']*df['puWeight']*leptonSF*wtag_SF
        weight_noWSF = np.ones(len(df['weight'])) if dataset=='Data' else df['weight']*df['puWeight']*leptonSF
        lumis       = {2016: 36., 2017: 41.5, 2018: 60.}
        cfg['lumi'] = 1 if dataset=='Data' else lumis[year]
        fullweight  = weight*cfg['lumi']
        fullweight_noWSF  = weight_noWSF*cfg['lumi']

        ### And fill the histograms
        output['met'].fill(dataset=dataset, pt=met_pt[selection].flatten(), weight=fullweight[selection])
        
        return output

    def postprocess(self, accumulator):
        return accumulator

In [10]:
fileset_2018 = {'TTJets_old':glob.glob('/hadoop/cms/store/user/mbryson/WH_hadronic/v0.2.4/TTJets_DiLept_Tune*/*.root')
                            +glob.glob('/hadoop/cms/store/user/mbryson/WH_hadronic/v0.2.4/TTJets_SingleLeptFromTbar_Tune*/*.root')
                            +glob.glob('/hadoop/cms/store/user/mbryson/WH_hadronic/v0.2.4/TTJets_SingleLeptFromT_Tune*/*.root'),
                'TTJets_new':glob.glob('/hadoop/cms/store/user/mbryson/WH_hadronic/v0.2.4/TTJets_DiLept_*/*.root')
                            +glob.glob('/hadoop/cms/store/user/mbryson/WH_hadronic/v0.2.4/TTJets_SingleLeptFromTbar_*/*.root')
                            +glob.glob('/hadoop/cms/store/user/mbryson/WH_hadronic/v0.2.4/TTJets_SingleLeptFromT_*/*.root')}

output_2018 = processor.run_uproot_job(fileset_2018,
                                    treename='Events',
                                    processor_instance=analysisProcessor(),
                                    executor=processor.futures_executor,
                                    executor_args={'workers': 12, 'function_args': {'flatten': False}},
                                    chunksize=500000,
                                 )

HBox(children=(HTML(value='Processing'), FloatProgress(value=0.0, max=597.0), HTML(value='')))

In [11]:
err_opts_rat = {
    'linestyle': 'none',
    'marker': '.',
    'markersize': 10.,
    'color':'#8AC926',
    'elinewidth': 1}

lineOverlayOpts = {
    'color': [('#1982C4'),('#F76F8E')]
}

outdir = plotDir+"baby_verification/"

In [12]:
def saveoverlayshape(hists, outdir, name):
    import re
    old = hists['TTJets_old']
    new = hists['TTJets_new']
    
    plt.rcParams.update({'font.size': 14,'axes.titlesize': 18,'axes.labelsize': 18,
                         'xtick.labelsize': 12,'ytick.labelsize': 12})
    fig, (ax, rax) = plt.subplots(nrows=2,ncols=1, figsize=(7,7),
    gridspec_kw={"height_ratios": (3, 1)}, sharex=True)
    fig.subplots_adjust(hspace=.07)
    hist.plot1d(hists, overlay="dataset",  ax=ax, clear=False, density = True, stack=False,
                line_opts = lineOverlayOpts, overflow = 'over')
    ax.set_yscale('log')
    ax.set_ylim(0, 1)
    ax.set_xlabel(None)
    leg = ax.legend()
    hist.plotratio(num=old.sum('dataset'), denom=new.sum('dataset'), ax=rax, clear = False,
                   error_opts = err_opts_rat, denom_fill_opts={}, guide_opts={}, 
                   unc='num', overflow = 'over')
    rax.set_ylim(0,2)
    rax.set_ylabel('Ratio')
    fig.savefig(os.path.join(outdir, "{}_log.png".format(name)))
    fig.savefig(os.path.join(outdir, "{}_log.pdf".format(name)))
    fig.clear()

In [13]:
hists_2018 = output_2018["met"]

saveoverlayshape(hists_2018,outdir,"MET_TTJets_2018_norebin")

new_pt18_bins1 = hist.Bin('pt',r'MET',40,200,1000)
hists_2018 = hists_2018.rebin('pt',new_pt18_bins1)
saveoverlayshape(hists_2018,outdir,"MET_TTJets_2018_40bin")

new_pt18_bins = hist.Bin('pt',r'MET',20,200,1000)
hists_2018 = hists_2018.rebin('pt',new_pt18_bins)
saveoverlayshape(hists_2018,outdir,"MET_TTJets_2018_20bin")

/cvmfs/cms.cern.ch/slc6_amd64_gcc700/external/py2-ipykernel/4.8.2-gnimlf2/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  
/home/users/ksalyer/CMSSW_10_2_9/src/tW_scattering/coffeaEnv/lib/python3.6/site-packages/coffea/hist/plot.py:357: RuntimeWarning: divide by zero encountered in true_divide
  rsumw = sumw_num / sumw_denom
/home/users/ksalyer/CMSSW_10_2_9/src/tW_scattering/coffeaEnv/lib/python3.6/site-packages/coffea/hist/plot.py:357: RuntimeWarning: invalid value encountered in true_divide
  rsumw = sumw_num / sumw_denom
/home/users/ksalyer/CMSSW_10_2_9/src/tW_scattering/coffeaEnv/lib/python3.6/site-packages/coffea/hist/plot.py:364: RuntimeWarning: divide by zero encountered in true_divide
  rsumw_err = np.abs(poisson_interval(rsumw, sumw2_num / sumw_denom**2) - rsumw)
/home/users/ksalyer/CMSSW_10_2_9/src/tW_scattering/coffeaEnv/lib/python3.6/site-packages/coffea/hist/

<Figure size 504x504 with 0 Axes>

<Figure size 504x504 with 0 Axes>

<Figure size 504x504 with 0 Axes>